### Preprocess data

In [ ]:
import os, glob
import numpy as np
import SimpleITK as sitk
import torch
import nibabel as nib
import shutil
import matplotlib.pyplot as plt
import torch.nn.functional as F
from skimage.exposure import equalize_hist
from einops.einops import rearrange
from ipywidgets import interact, IntSlider
from scipy.ndimage import generate_binary_structure, binary_opening, label
from skimage.measure import regionprops
from glob import glob
from scipy.ndimage import generate_binary_structure, binary_opening
from skimage.measure import label, regionprops
from skimage.morphology import erosion, disk, convex_hull_image

In [ ]:
# Función auxiliar para mostrar imágenes (se adapta a arreglos 2D o con canal único)
def display_debug(img, title="Imagen"):
    # Si es un arreglo enmascarado, se convierte a arreglo normal rellenando con 0
    if isinstance(img, np.ma.MaskedArray):
        img = img.filled(0)
    # Si la imagen tiene 3 dimensiones y el primer eje es de tamaño 1, mostramos ese canal
    if img.ndim == 3:
        if img.shape[0] == 1:
            img_to_show = img[0]
        else:
            img_to_show = img[0]  # O se podría elegir la del medio, según convenga
    else:
        img_to_show = img
    plt.figure()
    plt.imshow(img_to_show, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()


In [ ]:
def write_np(array, path):
    with open(path, 'wb') as f:
        np.save(f, array)

In [ ]:
import numpy as np

def preprocessor_4d(nifti, clip_value, debug=True):
    """
    Preprocesa un array 4D (t, z, x, y) aplicando:
      1. Clip: recortar los valores a [0, clip_value].
      2. Ecualización del histograma para cada imagen 2D (para cada t y z).
      3. Sustracción del mínimo global.
      4. Desplazar la distribución (restar 0.5) sin modificar los ceros.

    Además, en cada paso se verifica que no aparezcan valores extraños (NaN o Inf).
    Si se detectan, se identifican las slices problemáticas en la dimensión z y se eliminan
    de la imagen (en todos los frames t). Si tras la eliminación aún se encuentran problemas,
    se retorna la imagen original.

    Parámetros:
      nifti: np.ndarray
          Array 4D de entrada con dimensiones (t, z, x, y).
      clip_value: float
          Valor máximo para recortar la imagen.
      debug: bool
          Si es True, muestra imágenes intermedias (se muestra la imagen del primer frame y slice, t=0, z=0)
          y mensajes de aviso en caso de detectar valores extraños.

    Retorna:
      mdata: np.ndarray
          Array 4D preprocesado (con las slices z problemáticas eliminadas) o, en caso de no poder limpiar,
          la imagen original.
      z_map_local: list
          Lista con los índices de las slices z que se mantuvieron tras la limpieza.
    """

    # Función interna para verificar la validez del array
    def is_valid(arr):
        return np.isfinite(arr).all()

    def remove_problematic_z_slices(arr, debug, z_map):
        """
        Revisa cada slice z (para todos los t) y elimina aquellas en las que se encuentre algún valor
        extraño (NaN o Inf). Se espera que arr tenga forma (t, z, x, y).
        """
        t_dim, z_dim, _, _ = arr.shape
        good_z = []      # índices de slices sin problemas
        removed_z = []   # índices de slices a eliminar

        for z in range(z_dim):
            # Se evalúa la slice z en todos los t
            slice_z = arr[:, z, :, :]
            if np.isfinite(slice_z).all():
                good_z.append(z)
            else:
                removed_z.append(z)

        if debug and removed_z:
            print(f"Se detectaron valores extraños en las slices z: {removed_z}. Se eliminarán.")

        if len(good_z) == 0:
            if debug:
                print("Todas las slices z contienen valores extraños. No se puede limpiar.")
            return arr, z_map
        
        clean_arr = arr[:, good_z, :, :]
        new_z_map = [z_map[z] for z in good_z]

        # Se elimina la dimensión z problemática para TODOS los frames t
        return clean_arr, new_z_map

    t_dim, z_dim, x_dim, y_dim = nifti.shape
    z_map_local = list(range(z_dim))

    # --- Paso 1: Clip ---
    nifti_clipped = np.clip(nifti, 0, clip_value)
    if not is_valid(nifti_clipped):
        nifti_clipped, z_map_local = remove_problematic_z_slices(nifti_clipped, debug, z_map_local)

    if debug:
        display_debug(nifti_clipped[0, 0], "Después del clip (t=0, z=0)")

    # --- Paso 2: Ecualización del histograma para cada imagen 2D ---
    t_dim, z_dim, height, width = nifti_clipped.shape
    nifti_eq = np.empty((t_dim, z_dim, height, width), dtype=np.float32)
    for t in range(t_dim):
        for z in range(z_dim):
            # Se ecualiza cada imagen 2D individualmente.
            nifti_eq[t, z] = equalize_hist(
                nifti_clipped[t, z],
                nbins=20000,
                mask=(nifti_clipped[t, z] > 0)
            )
    if not is_valid(nifti_eq):
        nifti_eq, z_map_local = remove_problematic_z_slices(nifti_eq, debug, z_map_local)
        if not is_valid(nifti_eq):
            if debug:
                print("Aún se detectan valores extraños después de ecualizar el histograma y eliminación de slices. Se retorna la imagen original.")
            return nifti
    if debug:
        display_debug(nifti_eq[0, 0], "Después de ecualizar histograma (t=0, z=0)")

    # --- Paso 3: Sustracción del mínimo global ---
    global_min = np.min(nifti_eq)
    nifti_eq = nifti_eq - global_min
    if not is_valid(nifti_eq):
        nifti_eq = remove_problematic_z_slices(nifti_eq, debug)
        if not is_valid(nifti_eq):
            if debug:
                print("Aún se detectan valores extraños después de restar el mínimo y eliminación de slices. Se retorna la imagen original.")
            return nifti
    if debug:
        display_debug(nifti_eq[0, 0], "Después de restar el mínimo (t=0, z=0)")

    # --- Paso 4: Desplazar la distribución sin modificar los ceros (restar 0.5) ---
    mask = (nifti_eq > 0)
    # Se crea un array enmascarado; los ceros se mantienen inalterados.
    mdata = np.ma.masked_array(nifti_eq, mask=~mask)
    mdata = mdata - 0.5
    mdata.mask = np.ma.nomask  # Se "desenmascara" para continuar el procesamiento
    if not is_valid(mdata):
        mdata = remove_problematic_z_slices(mdata, debug)
        if not is_valid(mdata):
            if debug:
                print("Aún se detectan valores extraños después de desplazar la distribución y eliminación de slices. Se retorna la imagen original.")
            return nifti
    if debug:
        display_debug(mdata[0, 0], "Después de desplazar la distribución (t=0, z=0)")

    return mdata, z_map_local


In [ ]:
def generate_threshold_mask(data_2D, umbral=0.3, debug=False):
    """
    Genera una máscara de cerebro a partir del frame t=0 usando un umbral simple.
    - data: array (H, W)
    - umbral: valor de umbral para separar hueso de tejido (dependerá de tus datos)
    
    Devuelve:
      - mask: array binario (H, W) con True en las regiones (posible cerebro).
    """
    
    if debug:
        print("Seleccionado frame t=0.")
        print("Estadísticas de t0_image: min =", data_2D.min(), ", max =", data_2D.max())
        plt.figure()
        plt.title("Frame t=0 original")
        plt.imshow(data_2D, cmap='gray')
        plt.colorbar()
        plt.show()
    
    # 2. Aplicar el umbral: creamos una máscara con True donde el valor < umbral. 
    # False corresponde a la primera aproximación del cráneo.
    mask = data_2D < umbral
    if debug:
        print(f"Aplicado threshold: valores < {umbral} se vuelven True.")
        print("Valores únicos en la máscara tras threshold:", np.unique(mask))
        plt.figure()
        plt.title("Máscara después del threshold")
        plt.imshow(mask, cmap='gray')
        plt.colorbar()
        plt.show()
    
    # 3. Limpieza morfológica para eliminar ruido y rellenar huecos.
    # 3.1. Generamos la estructura 2D para la operación morfológica.
    struct_2d = generate_binary_structure(2, 1)
    
    # 3.2. Aplicamos binary opening (elimina pequeños objetos aislados).
    mask_opened = binary_opening(mask, structure=struct_2d, iterations=5)
    if debug:
        print("Después de binary_opening:")
        print("Valores únicos:", np.unique(mask_opened))
        plt.figure()
        plt.title("Máscara tras binary_opening")
        plt.imshow(mask_opened, cmap='gray')
        plt.colorbar()
        plt.show()
    
    # 3.3. Se identifican posibles calcificaciones y pasan a ser parte del cerebro y no del cráneo.
    # Para esto se identifican las regiones conectadas y se eliminan todas menos la más grande.
    inverted_mask = ~mask_opened

    # Etiquetamos las regiones conectadas
    labeled_false, num_false = label(inverted_mask, return_num=True)
    if debug:
        print(f"Número de regiones conectadas en la zona False: {num_false}")

    # Si hay más de una región False, mantenemos solo la mayor.
    if num_false > 1:
        regions = regionprops(labeled_false)
        # Seleccionamos la región con mayor área
        largest_region = max(regions, key=lambda r: r.area)
        largest_label = largest_region.label
        if debug:
            print(f"Se han encontrado más de una región False. La región más grande tiene área: {largest_region.area}")
        # Creamos una nueva máscara invertida donde solo se conserva la mayor región False
        new_inverted_mask = (labeled_false == largest_label)
        # La máscara final se obtiene invirtiendo esta nueva máscara: True es el cerebro y el background
        # y False en el resto (cráneo).
        mask_opened = ~new_inverted_mask
        if debug:
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 2, 1)
            plt.title("Inverted mask original")
            plt.imshow(inverted_mask, cmap='gray')
            plt.subplot(1, 2, 2)
            plt.title("Inverted mask solo mayor región")
            plt.imshow(new_inverted_mask, cmap='gray')
            plt.show()
    else:
        if debug:
            print("No se detectaron más de dos regiones False. No se aplica corrección.")
    
    # Forzar que los 10 píxeles del borde sean False
    borde = 6
    mask_opened[:, :borde] = False
    mask_opened[:, -borde:] = False
    
    # 3.4 Etiquetar las regiones conectadas en la máscara para eliminar el fondo (ahora se espera que quede solo el cerebro).
    labeled, num_labeled = label(mask_opened, return_num=True)
    regions = regionprops(labeled)
    if debug:
        print(f"Número de regiones conectadas en la zona True: {num_labeled}")
        if len(regions) > 0:
            print("Regiones encontradas en la zona True:")
            for reg in regions:
                print(f" - Etiqueta: {reg.label}, Área: {reg.area}, BBox: {reg.bbox}")
        else:
            print("No se encontraron regiones conectadas en la zona True.")
    
    if len(regions) == 0:
        if debug:
            print("No se detectó ninguna región. Se retorna la máscara tal como está.")
        final_mask = mask_opened
    else:
        # Ordenamos las regiones por área de mayor a menor.
        sorted_regions = sorted(regions, key=lambda r: r.area, reverse=True)
        if debug:
            print("Se unen las regiones a partir de la segunda:")
            for reg in sorted_regions[1:]:
                print(f" - Etiqueta: {reg.label}, Área: {reg.area}")
        # Inicializamos una máscara vacía (todos False) del mismo tamaño.
        final_mask = np.zeros_like(labeled, dtype=bool)
        # Unimos (con OR) todas las regiones desde la segunda en adelante.
        for region in sorted_regions[1:]:
            final_mask |= (labeled == region.label)
        if debug:
            plt.figure()
            plt.title("Máscara final (unión de regiones desde la segunda)")
            plt.imshow(final_mask, cmap='gray')
            plt.show()
            
    return final_mask

In [ ]:
def apply_masks_to_ct_volume(ct_volume, mask_folder, patient_id, composed_z_map, debug=False):
    """
    Aplica las máscaras almacenadas en mask_folder al volumen ct_volume usando la correspondencia de índices.
    
    Parámetros:
      ct_volume : numpy.ndarray
          Volumen CT con forma (t, new_z, x, y), donde new_z es la numeración tras filtrar slices.
      mask_folder : str
          Carpeta que contiene los archivos de máscara (.npy). Se espera que los archivos se nombren como:
          "case_{patient_id:02d}_{real_z+1:02d}.npy"
      patient_id : int or str
          Identificador del paciente.
      composed_z_map : dict
          Mapeo que relaciona new_z (índice funcional en ct_volume) -> real_z (índice original).
      debug : bool
          Si es True, se imprime información de depuración y se visualiza el resultado para cada slice.
    
    Devuelve:
      ct_volume : numpy.ndarray
          El volumen CT con las máscaras aplicadas en cada slice (para todos los frames temporales).
          Los píxeles donde la máscara es False se pondrán a 0.
    """
    # Convertir patient_id a cadena con dos dígitos si es numérico
    if isinstance(patient_id, int):
        patient_str = f"{patient_id:02d}"
    else:
        patient_str = patient_id

    num_new_z = ct_volume.shape[1]
    if debug:
        print(f"Aplicando máscaras a {num_new_z} slices (índices funcionales).")
    
    for new_z in range(num_new_z):
        # Obtener el índice real a partir del mapeo
        real_z = composed_z_map.get(new_z, None)
        if real_z is None:
            if debug:
                print(f"  Advertencia: No se encontró correspondencia para new_z={new_z}. Se omite.")
            continue
        
        # Construir el nombre del archivo usando el índice real (1-based)
        filename = f"case_{patient_str}_{real_z+1:02d}.npy"
        mask_path = os.path.join(mask_folder, filename)
        if not os.path.exists(mask_path):
            if debug:
                print(f"  Archivo de máscara no encontrado para new_z={new_z} (real_z={real_z}): {mask_path}")
            continue
        
        # Cargar la máscara (se espera que sea un array 2D de forma (x, y))
        mask = np.load(mask_path)
        if debug:
            print(f"Procesando máscara para new_z={new_z} (real_z={real_z}) desde {filename}")
            print(f"  Forma de la máscara: {mask.shape}, valores únicos: {np.unique(mask)}")
        
        # Verificar que la forma de la máscara coincida con la de un slice del CT
        if mask.shape != ct_volume.shape[2:]:
            if debug:
                print(f"  ERROR: La forma de la máscara {mask.shape} no coincide con la forma del slice {ct_volume.shape[2:]}")
            continue
        
        # Aplicar la máscara a todos los frames temporales para el slice new_z:
        # Donde la máscara es False, se anulan los píxeles.
        ct_volume[:, new_z, :, :] *= mask.astype(ct_volume.dtype)
        
        if debug:
            # Mostrar estadísticas del primer frame del slice y visualizarlo.
            frame = ct_volume[0, new_z, :, :]
            stats = (frame.min(), frame.max(), frame.mean())
            print(f"  Tras aplicar la máscara, primer frame stats (min, max, mean): {stats}")
            
            plt.figure(figsize=(6, 6))
            plt.imshow(frame, cmap='gray')
            plt.title(f"Masked slice: new_z={new_z} (real_z={real_z})")
            plt.colorbar()
            plt.show()
    
    return ct_volume

In [ ]:
def getSymmetricRepresentation_4d(ct_volume, debug=True):
    """
    Aplica la representación simétrica a un volumen 4D de CT_4DPWI con dimensiones (t, z, x, y).
    
    Para cada slice (índice z), se calcula la transformación a partir del frame t=0 y luego se
    aplica a todos los frames temporales de ese slice.
    
    Parámetros:
      ct_volume: np.ndarray
          Volumen 4D de entrada con forma (t, z, alto, ancho)
      debug: bool
          Si es True, se muestran imágenes intermedias usando display_debug.
    
    Retorna:
      registered_volume: np.ndarray
          Volumen 4D con las imágenes registradas, misma forma que ct_volume.
    """
    # Extraer dimensiones del volumen
    t_dim, z_dim, height, width = ct_volume.shape
    
    # Inicializar el volumen de salida
    registered_volume = np.zeros((t_dim, z_dim, height, width))
    
    # Diccionario para almacenar la transformación de cada slice (calculada a partir de t=0)
    transform_maps = {}
    
    # Para cada slice (z) calculamos la transformación usando el frame t=0
    for z in range(z_dim):
        ct_slice = ct_volume[0, z, :, :]
        if debug:
            display_debug(ct_slice, f"Slice z={z}, frame t=0 original")
        ct_slice_flipped = np.fliplr(ct_slice)
        if debug:
            display_debug(ct_slice_flipped, f"Slice z={z}, frame t=0 volteada")
        
        fixedImage = sitk.GetImageFromArray(ct_slice)
        movingImage = sitk.GetImageFromArray(ct_slice_flipped)
    
        # Configuramos el registro rígido con Elastix
        parameterMap = sitk.GetDefaultParameterMap("rigid")
        elastixImageFilter = sitk.ElastixImageFilter()
        elastixImageFilter.SetFixedImage(fixedImage)
        elastixImageFilter.SetMovingImage(movingImage)
        elastixImageFilter.LogToFileOn()
        elastixImageFilter.SetParameterMap(parameterMap)
        resultImage = elastixImageFilter.Execute()
        resultArray = sitk.GetArrayFromImage(resultImage)
        if debug:
            display_debug(resultArray, f"Resultado de registro para slice z={z} (frame t=0)")
    
        transform_maps[z] = elastixImageFilter.GetTransformParameterMap()
    
    # Aplicar la transformación calculada para cada slice a todos los frames temporales
    for z in range(z_dim):
        for t in range(t_dim):
            frame = ct_volume[t, z, :, :]
            # Opcional: mostrar debug para el primer frame de cada slice
            if debug and t == 0:
                display_debug(frame, f"Slice z={z}, frame t={t} original")
            frame_flipped = np.fliplr(frame)
            if debug and t == 0:
                display_debug(frame_flipped, f"Slice z={z}, frame t={t} volteado")
    
            movingFrame = sitk.GetImageFromArray(frame_flipped)
            frameResult = sitk.Transformix(movingFrame, transform_maps[z])
            resultFrame = sitk.GetArrayFromImage(frameResult)
            if debug and t == 0:
                display_debug(resultFrame, f"Slice z={z}, frame t={t} registrado")
            registered_volume[t, z, :, :] = resultFrame
    
    return registered_volume


In [ ]:
def smoothing_4d(array):
    """
    Aplica un suavizado temporal a un array 4D de forma (t, z, x, y)
    usando una convolución 1D a lo largo del eje temporal.
    
    El procedimiento es:
      1. Reordenar de (t, z, x, y) a (x, y, z, t).
      2. Para cada canal (slice, que equivale a z) se aplica conv1d a lo largo del eje temporal.
      3. Se reordena la salida a (t_out, z, x, y).
    
    Parámetros:
      array: np.ndarray de forma (t, z, x, y)
      
    Retorna:
      out_rearr: np.ndarray de forma (t_out, z, x, y), donde t_out depende de la convolución.
    """
    # Reordenar de (t, z, x, y) a (x, y, z, t)
    array_rearr = np.transpose(array, (2, 3, 1, 0))
    h, w, c, t = array_rearr.shape  # aquí h=x, w=y, c=z
    
    # Definir el kernel (suavizado ponderado)
    kernel_np = np.array([0.25, 0.5, 0.25])
    kernel_torch = torch.tensor(kernel_np, dtype=torch.float32, device='cuda')
    # Reorganizar para conv1d: (out_channels, in_channels, kernel_size) => (1, 1, 3)
    kernel_torch = kernel_torch.view(1, 1, -1)
    
    # Calcular la longitud de la salida real (stride=2, kernel=3, sin padding)
    output_len = (t - 3) // 2 + 1
    
    # Crear el array de salida en la forma (h, w, c, output_len)
    out = np.empty((h, w, c, output_len), dtype=np.float32)
    
    # Para cada canal (slice, es decir, para cada índice en c)
    for ch in range(c):
        # Extraer la secuencia temporal para ese canal: (h, w, t)
        channel = array_rearr[:, :, ch, :]
        # Aplanar el plano espacial: (h*w, t)
        channel_flat = channel.reshape(-1, t)
        # Convertir a tensor en CUDA
        channel_torch = torch.tensor(channel_flat, dtype=torch.float32, device='cuda')
        # Añadir la dimensión de canal para conv1d: (h*w, 1, t)
        channel_torch = channel_torch.unsqueeze(1)
        
        # Aplicar la convolución 1D a lo largo del eje temporal
        result = F.conv1d(channel_torch, kernel_torch, stride=2, padding=0)
        # result tiene forma (h*w, 1, output_len)
        result_np = result.squeeze(1).cpu().numpy()  # (h*w, output_len)
        # Volver a dar forma a (h, w, output_len)
        result_np = result_np.reshape(h, w, -1)
        out[:, :, ch, :] = result_np
        
    # Reordenar la salida a la forma (t_out, z, x, y)
    out_rearr = np.transpose(out, (3, 2, 0, 1))
    return out_rearr

In [ ]:
def save_smoothed_images(ct_volume, symmetric_volume, patient_id, z_map, output_folder):
    """
    Guarda la imagen resultante en el formato deseado.
    
    Parámetros:
      ct_volume: np.ndarray
          Volumen preprocesado original con forma (t, z, x, y).
      symmetric_volume: np.ndarray
          Volumen con la representación simétrica, mismo shape que ct_volume.
      patient_id: int o str
          Identificador del paciente (se usará en el nombre del archivo).
      z_map: dict
          Mapeo que relaciona el índice funcional (después de filtrado) con el índice real.
      output_folder: str
          Ruta de la carpeta donde se guardarán los archivos (p.ej., "/data/dev/perfu-net/data/train/CTP").
    
    El proceso es:
      1. Se aplica el suavizado a ambos volúmenes mediante smoothing_4d.
      2. Para cada slice (eje z), se crea un array de forma (t_out, 2, x, y),
         donde:
           - Canal 0: imagen original tras el suavizado.
           - Canal 1: imagen simétrica tras el suavizado.
      3. Si el slice es completamente negro (todos los valores 0), no se guarda.
      4. Se guarda cada slice en un archivo con nombre "case_{patient_id:02d}_{real_z+1:02d}.npy".
    """
    # Asegurarse de que la carpeta de salida exista
    os.makedirs(output_folder, exist_ok=True)
    
    # Aplicar el suavizado a ambos volúmenes.
    # Se supone que smoothing_4d está definida y recibe un array con shape (t, z, x, y),
    # devolviendo un array con shape (t_out, z, x, y), donde t_out = (t - 3) // 2 + 1.
    smoothed_original = smoothing_4d(ct_volume)
    smoothed_symmetric = smoothing_4d(symmetric_volume)
    
    # Extraer dimensiones (del volumen suavizado)
    t_out, z_dim, x, y = smoothed_original.shape
    print(f"Dimensión temporal tras suavizado: {t_out}")
    
    for z in range(z_dim):
        real_z = z_map[z]  # Índice real (original) para este slice
        # Extraer la secuencia temporal para el slice z (forma: (t_out, x, y))
        orig_slice = smoothed_original[:, z, :, :]
        sym_slice = smoothed_symmetric[:, z, :, :]
        
        # Crear un array combinado de forma (t_out, 2, x, y)
        combined = np.stack([orig_slice, sym_slice], axis=1)
        
        # Verificar si el slice combinado es completamente negro
        if np.all(combined == 0):
            print(f"Slice (real z={real_z}) completamente negro. No se guarda.")
            continue
        
        # Construir el nombre del archivo usando el índice real (1-based)
        filename = f"case_{int(patient_id):02d}_{real_z+1:02d}.npy"
        filepath = os.path.join(output_folder, filename)
        
        # Guardar el array
        np.save(filepath, combined)
        print(f"Guardado: {filepath}, shape: {combined.shape}")

    return smoothed_original, smoothed_symmetric

In [ ]:
def interactive_debug_ct4dpwi_nifti(file_path, patient_id, debug):
    """
    Carga un archivo NIfTI de CT_4DPWI con dimensiones (t, z, x, y), aplica el preprocesamiento,
    la representación simétrica y el suavizado, y permite depurar interactivamente mostrando:
      - Imagen original preprocesada.
      - Imagen volteada (np.fliplr).
      - Imagen registrada (simétrica).
      - Imagen suavizada (resultado de smoothing_4d).
    
    Se utiliza un mapeo simple para relacionar el índice temporal original con el del suavizado:
      smoothed_idx = t_idx // 2
    """

    output_folder = "/data/dev/perfu-net-1/data/train/CTP"
    
    # 1. Cargar la imagen NIfTI y obtener el array 4D
    ct_image = sitk.ReadImage(file_path)
    ct_volume = sitk.GetArrayFromImage(ct_image)
    print("Shape original (como lo devuelve sitk):", ct_volume.shape)
    
    # Se asume que ct_volume tiene forma (t, z, x, y)
    t_dim, z_dim, height, width = ct_volume.shape
    
    # Verificar y eliminar slices (componentes z) que tengan intensidades entre -25 y 0
    valid_z_indices = []
    print("\nEvaluando cada slice (componente z):")
    for z in range(z_dim):
        # Extraemos todos los frames para este slice z: forma (t, x, y)
        slice_data = ct_volume[:, z, :, :]
        min_intensity_z = slice_data.min()
        max_intensity_z = slice_data.max()
        print(f"  Slice z={z}: Intensidad mínima: {min_intensity_z}, Intensidad máxima: {max_intensity_z}")
        if min_intensity_z >= -25 and max_intensity_z <= 0:
            print(f"    --> Slice z={z} omitido (todos los valores entre -25 y 0)")
        else:
            valid_z_indices.append(z)
    
    if not valid_z_indices:
        print("No quedan slices válidos (todas las componentes z están entre -25 y 0). Se omite la imagen.")
        return
    
    # Filtramos ct_volume a solo los slices válidos
    ct_volume = ct_volume[:, valid_z_indices, :, :]
    # Creamos un mapeo: new_z -> old_z
    z_map = { new_z: old_z for new_z, old_z in enumerate(valid_z_indices) }
    z_dim = ct_volume.shape[1]  # Actualizar el número de slices

    # 2. Preprocesar el volumen 4D (esta función retorna también un mapeo de los slices que quedaron)
    ct_volume, preprocessor_z_map = preprocessor_4d(ct_volume, 500, debug=False)
    
    # Componer el mapeo: 
    #   preprocessor_z_map mapea: new_z_pp -> índice en el ct_volume antes de preprocessor
    #   z_map mapea: índice en el ct_volume inicial -> índice original
    # Componemos para que: composed_z_map[new_z_pp] = z_map[ preprocessor_z_map[new_z_pp] ]
    composed_z_map = {}
    for new_z_pp, intermed_z in enumerate(preprocessor_z_map):
        original_z = z_map[intermed_z]
        composed_z_map[new_z_pp] = original_z

    # Actualizar z_dim según el volumen preprocesado
    z_dim = ct_volume.shape[1]
    
    # 3.1. Generar la máscara de umbral para el cerebro
    # Usamos composed_z_map para el naming: la máscara se guardará con el índice original.
    output_mask_folder = "/data/dev/perfu-net-1/data/train/SKULL_MASK"
    os.makedirs(output_mask_folder, exist_ok=True)
    for new_z in range(z_dim):
        real_z = composed_z_map[new_z]  # Índice original
        print(f"\nGenerando máscara para slice new_z={new_z} (original z={real_z})")
        # Se extrae el slice correspondiente: se toma el frame t=0 y el slice new_z
        slice_2d = ct_volume[0, new_z, :, :]  # (H, W)
        # Aquí se llama a la función de umbralado para un slice 2D;
        # suponemos que generate_threshold_mask procesa un slice 2D y devuelve la máscara.
        skull_mask = generate_threshold_mask(slice_2d, umbral=0.3, debug=False)
        
        filename = f"case_{int(patient_id):02d}_{real_z+1:02d}.npy"
        saving_path = os.path.join(output_mask_folder, filename)
        np.save(saving_path, skull_mask)
        if debug:
            print(f"Guardada máscara en: {saving_path}")
            
    ct_volume = apply_masks_to_ct_volume(ct_volume, output_mask_folder, patient_id, composed_z_map, debug=debug)

    # 3. Calcular la representación simétrica
    symmetric_volume = getSymmetricRepresentation_4d(ct_volume, debug=debug)
    
    # 5. Guardar imágenes suavizadas, pasando el mapeo compuesto para que los nombres usen el índice original.
    ct_volume_smoothed, symmetric_volume_smoothed = save_smoothed_images(ct_volume, symmetric_volume, patient_id, composed_z_map, output_folder)

    
    def show_frame(t_idx, z_idx):
        """
        Para un frame temporal t_idx y slice z_idx, muestra:
          - La imagen original preprocesada.
          - La imagen volteada (np.fliplr).
          - La imagen registrada (simétrica).
          - La imagen suavizada (resultado de smoothing_4d).
        
        Se mapea el índice t_idx al del suavizado usando:
          smoothed_idx = t_idx // 2
        """
        original = ct_volume[t_idx, z_idx, :, :]
        flipped = np.fliplr(original)
        registered = symmetric_volume[t_idx, z_idx, :, :]
        smoothed_idx = t_idx // 2  # Mapeo simple para relacionar ambos tiempos.
        t_smoothed = ct_volume_smoothed.shape[0]
        # Asegurarse de que el índice no exceda el rango del volumen suavizado:
        if smoothed_idx >= t_smoothed:
            smoothed_idx = t_smoothed - 1
        smoothed = symmetric_volume_smoothed[smoothed_idx, z_idx, :, :]
        
        # Mostrar en 4 subplots (2 filas x 2 columnas)
        fig, axs = plt.subplots(2, 2, figsize=(12, 10))
        axs = axs.flatten()
        
        axs[0].imshow(original, cmap='gray')
        axs[0].set_title(f"t={t_idx}, z={z_idx} original")
        axs[0].axis('off')
        
        axs[1].imshow(flipped, cmap='gray')
        axs[1].set_title(f"t={t_idx}, z={z_idx} volteado")
        axs[1].axis('off')
        
        axs[2].imshow(registered, cmap='gray')
        axs[2].set_title(f"t={t_idx}, z={z_idx} registrado")
        axs[2].axis('off')
        
        axs[3].imshow(smoothed, cmap='gray')
        axs[3].set_title(f"smoothed (t={smoothed_idx}, z={z_idx})")
        axs[3].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    # 5. Crear sliders interactivos para elegir el frame (t) y el slice (z)
    interact(show_frame,
             t_idx=IntSlider(min=0, max=t_dim-1, step=1, value=0, description="Tiempo (t)"),
             z_idx=IntSlider(min=0, max=z_dim-1, step=1, value=0, description="Slice (z)"))


In [ ]:
# Ruta raíz donde se encuentran los casos de TRAINING
training_path = '/data/ISLES-2018/TRAINING'
# Ruta de la carpeta donde se almacenan los pacientes ya procesados
output_dir = '/data/dev/perfu-net-1/data/train/CTP'

# Iterar sobre cada carpeta de paciente (se asume el patrón "case_XX")
for case_folder in glob(os.path.join(training_path, 'case_*')):
    # Extraer el id del paciente (por ejemplo, "12" de "case_12")
    # Asegurarse de que el patient_id tenga dos dígitos (añadir 0 a la izquierda si es necesario)
    patient_id = case_folder.split(os.sep)[-1].split('_')[1]
    if len(patient_id) == 1:
        patient_id = "0" + patient_id
    
    # Verificar si este paciente ya fue procesado.
    # Se busca cualquier archivo cuyo nombre contenga "case_{patient_id}" en output_dir.
    processed_files = glob(os.path.join(output_dir, f"case_{patient_id}_*.npy"))
    if processed_files:
        print(f"Paciente {patient_id} ya procesado. Saltando.")
        continue

    print(f"Procesando paciente {patient_id} en {case_folder}")
    
    # Buscar de forma recursiva archivos .nii dentro de cada carpeta del caso
    nii_files = glob(os.path.join(case_folder, '**', '*.nii'), recursive=True)
    for file_path in nii_files:
        # Sólo procesar los archivos que contengan "CT_4DPWI" en su ruta/nombre
        if 'CT_4DPWI' in file_path:
            print(f"  Procesando archivo: {file_path}")
            # Se llama a la función interactiva, pasando file_path, patient_id y debug=False
            interactive_debug_ct4dpwi_nifti(file_path, patient_id, False)